In [2]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule, EmbedMultipleConfs, MMFFOptimizeMoleculeConfs
import pickle
from tqdm import tqdm

In [3]:
def mol_to_morgan(mol, radius=3, n_bits=2048, chiral=False, features=False):
    from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
    return np.frombuffer(
        GetMorganFingerprintAsBitVect(
            mol, radius=radius, nBits=n_bits, useChirality=chiral, useFeatures=features
        ).ToBitString().encode(), "u1") - ord("0")

def mol_to_atoms_coords(
    m, hydrogenate=True, adj_matrix=False, do_morgan=False, optimize=False, numConfs=1, numThreads=1
):
    # Check if m is a string, and if so, convert it to an RDKit molecule object
    if isinstance(m, str):
        m = Chem.MolFromSmiles(m)
        if m is None:
            return None  # Could not parse the SMILES string into a molecule

    m3 = Chem.AddHs(m) if hydrogenate else m
    if optimize and hydrogenate:
        try:
            EmbedMultipleConfs(
                m3, numConfs=numConfs, pruneRmsThresh=0.125, randomSeed=0xF00D, numThreads=numThreads
            )
            opt = MMFFOptimizeMoleculeConfs(m3, mmffVariant="MMFF94s", numThreads=numThreads)
            opt = np.array(opt)
            converged = opt[:, 0] == 0
            if converged.any():
                lowest_eng_conformer = np.argmin(opt[converged][:, 1])
                lowest_energy = opt[converged][lowest_eng_conformer, 1]
                best_conf = np.arange(opt.shape[0])[converged][lowest_eng_conformer]
                c0 = m3.GetConformer(id=int(best_conf))
            else:
                # Fallback in case optimization did not converge
                c0 = m3.GetConformer()
                lowest_energy = None
        except Exception as Ex:
            # Fallback in case of an exception during embedding or optimization
            EmbedMolecule(m3, randomSeed=0xF00D)
            c0 = m3.GetConformers()[-1]
            lowest_energy = None
    else:
        EmbedMolecule(m3, randomSeed=0xF00D)
        c0 = m3.GetConformers()[-1]

    coords = c0.GetPositions()
    atoms = np.array([atom.GetAtomicNum() for atom in m3.GetAtoms()], dtype=np.uint8)

    to_return = [atoms, coords]

    if adj_matrix:
        to_return.append(Chem.GetAdjacencyMatrix(m3))

    if do_morgan:
        to_return.append(mol_to_morgan(m, radius=3, n_bits=2048, chiral=False))

    if optimize:
        to_return.append(lowest_energy)

    return tuple(to_return)

In [10]:
# Load the SMILES data
df_train = pd.read_csv('data/train_guacamol.smiles', header=None, names=['smiles'])
df_valid = pd.read_csv('data/valid_guacamol.smiles', header=None, names=['smiles'])
df_test = pd.read_csv('data/test_guacamol.smiles', header=None, names=['smiles'])

In [6]:
# Convert the lists of dictionaries to DataFrames and save as CSV files
def save_as_csv(preprocessed_data, filename):
    df = pd.DataFrame(preprocessed_data)
    # If 'coords' and 'atoms' are lists, they will be saved as strings in the CSV. You might need to parse them when reading.
    df.to_csv(filename, index=False)

In [14]:
# load pickle file and safe it as .csv
with open('train_guacamol.pkl', 'rb') as f:
    train_data = pickle.load(f)
save_as_csv(train_data, 'train_guacamol.csv')

with open('valid_guacamol.pkl', 'rb') as f:
    valid_data = pickle.load(f)
save_as_csv(valid_data, 'valid_guacamol.csv')

with open('test_guacamol.pkl', 'rb') as f:
    test_data = pickle.load(f)
save_as_csv(test_data, 'test_guacamol.csv')

In [12]:
def preprocess_smiles_data(smiles_data, partition_name):
    preprocessed_data = []
    for smi in tqdm(smiles_data):
        try:
            atoms, coords = mol_to_atoms_coords(smi)
            preprocessed_data.append({
                'smiles': smi,
                'atoms': atoms,
                'coords': coords,
                'partition': partition_name  # Add partition key
            })
        except Exception as e:
            print(f"Failed to process {smi}: {str(e)}")

    return preprocessed_data

# Pass the partition name explicitly when calling this function
with open('train_guacamol.pkl', 'wb') as f:
    pickle.dump(preprocess_smiles_data(df_train['smiles'], 'train'), f)

with open('valid_guacamol.pkl', 'wb') as f:
    pickle.dump(preprocess_smiles_data(df_valid['smiles'], 'valid'), f)

with open('test_guacamol.pkl', 'wb') as f:
    pickle.dump(preprocess_smiles_data(df_test['smiles'], 'test'), f)


 64%|██████▎   | 50634/79568 [2:14:25<2796:46:07, 347.98s/it]

Failed to process CCC(C=CC(C)C1CCC2C3C4=CC5(CC(Cl)CCC5(C)C3CCC12C)NC(=O)N4)C(C)C: 'NoneType' object has no attribute 'GetPositions'


 64%|██████▍   | 51142/79568 [2:15:13<42:49, 11.06it/s][08:23:05] UFFTYPER: Unrecognized atom type: Se2+2 (14)
[08:23:05] UFFTYPER: Unrecognized atom type: Se2+2 (14)
 65%|██████▍   | 51520/79568 [2:15:45<56:46,  8.23it/s]  

Failed to process CCCNC(=O)CCC(=O)OC(C(=O)N1CSC(C)(C)C1C(=O)NC(C)(C)C)C(Cc1ccccc1)NC(=O)COc1c(C)cccc1C: 'NoneType' object has no attribute 'GetPositions'


 66%|██████▋   | 52729/79568 [2:17:30<42:14, 10.59it/s]

...

 99%|█████████▊| 235469/238706 [12:17:00<2:46:49,  3.09s/it]

Failed to process COc1cc2cc(c1O)-c1cc(ccc1O)CCC(OS(=O)(=O)O)CCCC2: 'NoneType' object has no attribute 'GetPositions'


 99%|█████████▉| 235725/238706 [12:17:27<04:06, 12.11it/s]  

Failed to process COC(=O)CC(NC(=O)OC(C)(C)C)C(=O)N(Cc1ccccc1)C1(C(=O)NCc2ccccc2)CCN(Cc2ccccc2)C1: 'NoneType' object has no attribute 'GetPositions'


100%|██████████| 238706/238706 [12:43:47<00:00,  5.21it/s]


In [4]:
# load pk file and display first 5 entrys
with open('train_guacamol.pkl', 'rb') as f:
    train_data = pickle.load(f)
print(train_data[:5])

# load pk file and display first 5 entrys
with open('valid_guacamol.pkl', 'rb') as f:
    valid_data = pickle.load(f)
print(valid_data[:5])

# load pk file and display first 5 entrys
with open('test_guacamol.pkl', 'rb') as f:
    test_data = pickle.load(f)
print(test_data[:5])

[{'smiles': 'CCC(C)(C)Br', 'atoms': array([ 6,  6,  6,  6,  6, 35,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
      dtype=uint8), 'coords': array([[ 1.73995939, -0.41652801, -0.33496531],
       [ 0.77919738, -0.37885212,  0.79409621],
       [-0.59915176,  0.09005053,  0.43215675],
       [-1.2439719 , -0.79659656, -0.61070348],
       [-0.65889248,  1.49995473, -0.0469612 ],
       [-1.69814093, -0.07384639,  2.03462616],
       [ 2.25522228, -1.404456  , -0.38616722],
       [ 2.57259291,  0.32551001, -0.17056892],
       [ 1.2720485 , -0.27115557, -1.32946089],
       [ 0.661645  , -1.41826167,  1.20853113],
       [ 1.204467  ,  0.21590541,  1.62506238],
       [-1.05566783, -0.36153166, -1.62369487],
       [-2.33335852, -0.85974912, -0.49246425],
       [-0.75625327, -1.79781899, -0.56980021],
       [-1.71496358,  1.84056761,  0.02737312],
       [-0.35748139,  1.60001995, -1.12210047],
       [-0.06725082,  2.20678784,  0.56504107]]), 'partition': 'train'}, {'smiles': 'CCCN(

In [7]:
train_valid = train_data + valid_data
save_as_csv(train_valid, 'train_valid_guacamol.csv')

# save train_valid as pk file
with open('train_valid_guacamol.pkl', 'wb') as f:
    pickle.dump(train_valid, f)

train_valid_test = train_valid + test_data
save_as_csv(train_valid_test, 'train_valid_test_guacamol.csv')

# save train_valid_test as pk file
with open('train_valid_test_guacamol.pkl', 'wb') as f:
    pickle.dump(train_valid_test, f)